In [1]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets

In [2]:
def t_test(a, b):
    """
    two-sided Student's t test
    """
    mean1, mean2 = np.mean(a), np.mean(b)
    s1, s2 = np.std(a, ddof=1), np.std(b, ddof=1)
    n1, n2 = len(a), len(b)
    
    df = n1+n2-2
    sp = np.sqrt(((n1-1)*(s1**2)+(n2-1)*(s2**2))/df)
    t = (mean1-mean2)/(sp*np.sqrt(1/n1+1/n2))
    p_value = (1-stats.t.cdf(abs(t), df))*2
    return t, p_value

def approx_t_test(a, b):
    """
    two-sided t test approximated by z test
    """
    mean1, mean2 = np.mean(a), np.mean(b)
    s1, s2 = np.std(a, ddof=1), np.std(b, ddof=1)
    n1, n2 = len(a), len(b)
    
    se1, se2 = s1/np.sqrt(n1), s2/np.sqrt(n2)
    se = np.sqrt(se1**2+se2**2)
    z = (mean1-mean2)/se
    p_value = (1-stats.norm.cdf(abs(z)))*2
    return z, p_value

def welch_test(a, b):
    """
    two-sided Welch's t test
    """
    mean1, mean2 = np.mean(a), np.mean(b)
    s1, s2 = np.std(a, ddof=1), np.std(b, ddof=1)
    n1, n2 = len(a), len(b)
    
    sn1, sn2 = s1**2/n1, s2**2/n2
    df = (sn1+sn2)**2/(sn1**2/(n1-1)+sn2**2/(n2-1))
    t = (mean1-mean2)/np.sqrt(sn1+sn2)
    p_value = (1-stats.t.cdf(abs(t), df))*2
    return t, p_value

def simulate(mu1,
             mu2,
             sigma1,
             sigma2, 
             n1,
             n2,
             n_simul=1_000,
             tests=[approx_t_test, t_test, welch_test]
            ):
    """
    Draw random samples from normal distribution and apply the tests
    """
    results = {}
    for _ in range(n_simul):
        sample1 = np.random.normal(mu1, sigma1, n1)
        sample2 = np.random.normal(mu2, sigma2, n2)
        for test in tests:
            stat, p_value = test(sample1, sample2)
            res = results.setdefault(test.__name__, {'stats': [], 'p_values': []})
            res['stats'].append(stat)
            res['p_values'].append(p_value)
    return results

def plot(results, alpha=0.05):
    """
    Plot distributions of test statistics and p values
    """
    for test in results:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        fig.suptitle(test)
        ax1.set_title('Distribution of test statistic')
        sns.distplot(results[test]['stats'], kde=False, norm_hist=True, ax=ax1, fit=stats.norm)
        ax2.set_title('Distribution of p value')
        sns.distplot(results[test]['p_values'], kde=False, norm_hist=True, ax=ax2)
        rejected = round((np.array(results[test]['p_values']) < alpha).mean() * 100, 1)
        text = f'H0 rejected in {rejected}% cases (alpha={alpha})'
        ax2.text(0.99, 0.95, text, transform=ax2.transAxes, horizontalalignment='right')
        
def run(mu1, mu2, sigma1, sigma2, n1, n2):
    plot(simulate(mu1, mu2, sigma1, sigma2, n1, n2))
    
simulation = widgets.interactive(
    run, 
    {'manual': True},
    mu1=widgets.FloatSlider(min=-5, max=5, step=0.1, value=0),
    mu2=widgets.FloatSlider(min=-5, max=5, step=0.1, value=0),
    sigma1=widgets.FloatSlider(min=0.1, max=5, step=0.1, value=1),
    sigma2=widgets.FloatSlider(min=0.1, max=5, step=0.1, value=1),
    n1=widgets.IntSlider(min=10, max=10_000, step=1, value=30),
    n2=widgets.IntSlider(min=10, max=10_000, step=1, value=30)
)

### Run simulation interactively

In [3]:
simulation

interactive(children=(FloatSlider(value=0.0, description='mu1', max=5.0, min=-5.0), FloatSlider(value=0.0, des…